In [1]:
import pandas as pd

In [14]:
gl = pd.read_csv('./data/game_logs.csv')
# gl.head()

In [10]:
print(gl.shape)

(171907, 161)


In [9]:
gl.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 171907 entries, 0 to 171906
Columns: 161 entries, date to acquisition_info
dtypes: float64(77), int64(6), object(78)
memory usage: 860.5 MB


In [16]:
for dtype in ['float64','int64','object']:
    selected_dtype = gl.select_dtypes(include = [dtype])
    mean_usage_b = selected_dtype.memory_usage(deep=True).mean()
    mean_usage_mb = mean_usage_b/1024**2
    print ('平均内存占用',dtype,mean_usage_mb)

平均内存占用 float64 1.2947326073279748
平均内存占用 int64 1.1241934640066964
平均内存占用 object 9.514454069016855


In [17]:
import numpy as np

In [18]:
int_types = ['uint8','int8','int16','int32','int64']
for it in int_types:
    print (np.iinfo(it))

Machine parameters for uint8
---------------------------------------------------------------
min = 0
max = 255
---------------------------------------------------------------

Machine parameters for int8
---------------------------------------------------------------
min = -128
max = 127
---------------------------------------------------------------

Machine parameters for int16
---------------------------------------------------------------
min = -32768
max = 32767
---------------------------------------------------------------

Machine parameters for int32
---------------------------------------------------------------
min = -2147483648
max = 2147483647
---------------------------------------------------------------

Machine parameters for int64
---------------------------------------------------------------
min = -9223372036854775808
max = 9223372036854775807
---------------------------------------------------------------



In [19]:
def mem_usage(pandas_obj):
    if isinstance(pandas_obj,pd.DataFrame):
        usage_b = pandas_obj.memory_usage(deep=True).sum()
    else:
        usage_b = pandas_obj.memory_usage(deep=True)
    usage_mb = usage_b/1024**2
    return '{:03.2f} MB'.format(usage_mb)

In [20]:
gl_int = gl.select_dtypes(include = ['int64'])
coverted_int = gl_int.apply(pd.to_numeric,downcast='unsigned')
print (mem_usage(gl_int))
print (mem_usage(coverted_int))

7.87 MB
1.48 MB


In [21]:
gl_float = gl.select_dtypes(include=['float64'])
converted_float = gl_float.apply(pd.to_numeric,downcast='float')
print(mem_usage(gl_float))
print(mem_usage(converted_float))

100.99 MB
50.49 MB


In [22]:
optimized_gl = gl.copy()
optimized_gl[coverted_int.columns] = coverted_int
optimized_gl[converted_float.columns] = converted_float
print(mem_usage(gl))
print(mem_usage(optimized_gl))

860.50 MB
803.61 MB


In [24]:
gl_obj = gl.select_dtypes(include = ['object']).copy()
print(gl_obj.describe())

       day_of_week  v_name v_league  h_name h_league day_night  \
count       171907  171907   171907  171907   171907    140150   
unique           7     148        7     148        7         2   
top            Sat     CHN       NL     CHN       NL         D   
freq         28891    8870    88866    9024    88867     82724   

                   completion forefeit protest park_id  ... h_player_6_id  \
count                     116      145     180  171907  ...        140838   
unique                    116        3       5     245  ...          4774   
top     19820711,CHI11,5,5,54        H       V   STL07  ...      grimc101   
freq                        1       69      90    7022  ...           427   

       h_player_6_name h_player_7_id h_player_7_name h_player_8_id  \
count           140838        140838          140838        140838   
unique            4720          5253            5197          4760   
top      Charlie Grimm      grimc101   Charlie Grimm      lopea102   
fre

In [25]:
dow = gl_obj.day_of_week
print(dow.head())

0    Thu
1    Fri
2    Sat
3    Mon
4    Tue
Name: day_of_week, dtype: object


In [26]:
dow_cat = dow.astype('category')
print(dow_cat.head())

0    Thu
1    Fri
2    Sat
3    Mon
4    Tue
Name: day_of_week, dtype: category
Categories (7, object): [Fri, Mon, Sat, Sun, Thu, Tue, Wed]


In [28]:
print(dow_cat.head(10).cat.codes)

0    4
1    0
2    2
3    1
4    5
5    4
6    2
7    2
8    1
9    5
dtype: int8


In [29]:
print(mem_usage(dow))
print(mem_usage(dow_cat))

9.84 MB
0.16 MB


In [31]:
converted_obj = pd.DataFrame()

for col in gl_obj.columns:
    num_unique_values = len(gl_obj[col].unique())
    num_total_values = len(gl_obj[col])
    if num_unique_values / num_total_values < 0.5:
        converted_obj.loc[:,col] = gl_obj[col].astype('category')
    else:
        converted_obj.loc[:,col] = gl_obj[col]

In [32]:
print(mem_usage(gl_obj))
print(mem_usage(converted_obj))

751.64 MB
51.67 MB


In [34]:
date = optimized_gl.date
print(date[:5])

0    18710504
1    18710505
2    18710506
3    18710508
4    18710509
Name: date, dtype: uint32


In [35]:
print(mem_usage(date))

0.66 MB


In [36]:
optimized_gl['date'] = pd.to_datetime(date,format='%Y%m%d')
print (mem_usage(optimized_gl['date']))

1.31 MB


In [37]:
print(optimized_gl['date'][:5])

0   1871-05-04
1   1871-05-05
2   1871-05-06
3   1871-05-08
4   1871-05-09
Name: date, dtype: datetime64[ns]
